In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForImageTextToText, GenerationConfig, pipeline
import torch
import torch.nn as nn

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
).to(device="mps")
model.eval()

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


In [5]:
prompt = "Complete successive parts of a sentence given one word at a time:"
target = "Now is the time for all good men to come to the aid of their country ."
targetVec = target.split()

prompt += ' '
prompt += targetVec[0]
inputs = tokenizer(prompt, return_tensors="pt").to(model.device) 
with torch.no_grad():
    raw_outputs = model(**inputs)
prob = torch.softmax(raw_outputs.logits, dim=-1)
###################################################
# Mhammad's code adds---put after computing raw outputs
max = torch.max(raw_outputs.logits[0, -1, :])
exp1 = torch.exp(raw_outputs.logits[0, -1, :]-max)   # deltas
exp2 = torch.exp(exp1) - 1
double_sf_probs = exp2 / torch.sum(exp2)
topValues, top_id = torch.topk(prob[0, -1,:], 50)

print("Single,doube softmaxed probs from raw output logits on deltas for prompt: ", prompt)
for t_id in top_id:
    t_str = tokenizer.decode(t_id.item()).strip()
    #print(f"{t_str}: {raw_outputs.logits[0, -1, :][t_id].item()}; {prob[0, -1,:][t_id].item():.4f}; {double_sf_probs[t_id].item():.4f}")
    print(f"{t_str}: {raw_outputs.logits[0, -1, :][t_id].item()}")

print("Press Enter to continue...")
input()

Single,doube softmaxed probs from raw output logits on deltas for prompt:  Complete successive parts of a sentence given one word at a time: Now
,: 20.25
I: 16.625
you: 16.625
the: 16.375
is: 15.75
suppose: 15.4375
that: 15.25
consider: 14.875
we: 14.625
: 14.5625
let: 14.3125
think: 14.125
it: 14.0
imagine: 13.9375
: 13.8125
: 13.6875
,: 13.625
choose: 13.5625
.: 13.375
.: 13.3125
write: 13.3125
a: 13.25
in: 13.125
...: 13.125
to: 13.0625
...: 13.0625
your: 13.0
...: 13.0
for: 12.875
You: 12.875
complete: 12.875
what: 12.8125
:: 12.6875
assume: 12.6875
,: 12.625
here: 12.625
.: 12.5
find: 12.5
: 12.4375
...: 12.4375
here: 12.125
...: 12.125
i: 12.0625
each: 12.0625
,...: 12.0625
(: 12.0
compute: 12.0
[: 11.9375
take: 11.9375
divide: 11.9375
Press Enter to continue...


'Now'

In [10]:
with open('feed.txt', 'r') as file:
    text = file.read()
    lowercase_text = text.lower()
    words = lowercase_text.split()
    print(f"Total Tokens in the text = {len(target)}")
    file.close()

Total Tokens in the text = 6383


In [11]:
target


[['Once', 'again'],
 ['to'],
 ['Zelda'],
 [],
 [],
 ['Then', 'wear', 'the', 'gold', 'hat,', 'if', 'that', 'will', 'move', 'her;'],
 ['If', 'you', 'can', 'bounce', 'high,', 'bounce', 'for', 'her', 'too,'],
 ['Till', 'she', 'cry', '“Lover,', 'gold-hatted,', 'high-bouncing', 'lover,'],
 ['I', 'must', 'have', 'you!”'],
 [],
 ['Thomas', 'Parke', 'd’Invilliers'],
 [],
 [],
 ['I'],
 [],
 ['In',
  'my',
  'younger',
  'and',
  'more',
  'vulnerable',
  'years',
  'my',
  'father',
  'gave',
  'me',
  'some',
  'advice'],
 ['that',
  'I’ve',
  'been',
  'turning',
  'over',
  'in',
  'my',
  'mind',
  'ever',
  'since.'],
 [],
 ['“Whenever',
  'you',
  'feel',
  'like',
  'criticizing',
  'anyone,”',
  'he',
  'told',
  'me,',
  '“just'],
 ['remember',
  'that',
  'all',
  'the',
  'people',
  'in',
  'this',
  'world',
  'haven’t',
  'had',
  'the',
  'advantages'],
 ['that', 'you’ve', 'had.”'],
 [],
 ['He',
  'didn’t',
  'say',
  'any',
  'more,',
  'but',
  'we’ve',
  'always',
  'been',
  '

In [21]:
target=[]
with open('feed.txt', 'r') as file:
    lines = file.readlines()
    for words in lines:
        lst = words.strip().split()
        target.extend(lst)


In [23]:
target=[]
with open('feed.txt', 'r') as file:
    lines = file.readlines()
    for words in lines:
        lst = words.strip().split()
        target.extend(lst)
    print(f"Total Tokens in the text = {len(target)}")
    file.close()

Total Tokens in the text = 48189


In [22]:
target

['Once',
 'again',
 'to',
 'Zelda',
 'Then',
 'wear',
 'the',
 'gold',
 'hat,',
 'if',
 'that',
 'will',
 'move',
 'her;',
 'If',
 'you',
 'can',
 'bounce',
 'high,',
 'bounce',
 'for',
 'her',
 'too,',
 'Till',
 'she',
 'cry',
 '“Lover,',
 'gold-hatted,',
 'high-bouncing',
 'lover,',
 'I',
 'must',
 'have',
 'you!”',
 'Thomas',
 'Parke',
 'd’Invilliers',
 'I',
 'In',
 'my',
 'younger',
 'and',
 'more',
 'vulnerable',
 'years',
 'my',
 'father',
 'gave',
 'me',
 'some',
 'advice',
 'that',
 'I’ve',
 'been',
 'turning',
 'over',
 'in',
 'my',
 'mind',
 'ever',
 'since.',
 '“Whenever',
 'you',
 'feel',
 'like',
 'criticizing',
 'anyone,”',
 'he',
 'told',
 'me,',
 '“just',
 'remember',
 'that',
 'all',
 'the',
 'people',
 'in',
 'this',
 'world',
 'haven’t',
 'had',
 'the',
 'advantages',
 'that',
 'you’ve',
 'had.”',
 'He',
 'didn’t',
 'say',
 'any',
 'more,',
 'but',
 'we’ve',
 'always',
 'been',
 'unusually',
 'communicative',
 'in',
 'a',
 'reserved',
 'way,',
 'and',
 'I',
 'underst

In [1]:
with open('output.txt', 'r') as file:
            line_count = sum(1 for line in file)

In [2]:
line_count

41428